<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## Tracking Invitation Homes Quarterly Activity From the Properties V2 Endpoint

In this notebook, we will analyze Invitation Homes 2024 quarterly activity in the US across in four key metrics
- Acquisitions
- Rental Listings
- Rent Rate
- Inventory

The notebook is broken up into the following sections:
1. Import required packages and setup the Parcl Labs API key and API headers
2. Leverage the V2 Prop Endpoint for the Point in Time Metrics (Aquisitions, Rental Listings and Rent Rate)
3. Leverage both the V1 Prop Endpoint for the Quarterly Inventory

**Reminders:**

- You can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

- To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`. 
- To run this notebook at scale and download data for multiple markets and endpoints, you will need to upgrade your Parcl Labs API account from free to starter to get additional credits. You can easily upgrade at any time by visiting your [Parcl Labs dashboard](https://dashboard.parcllabs.com/login), clicking "Upgrade Now" ($99, no commitment). This will unlock more credits immediately.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/investor_analytics/invh_yoy_change.ipynb)

### 1. Import required packages and setup the Parcl Labs API key and API headers

In [1]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: parcllabs
    Found existing installation: parcllabs 1.11.2
    Uninstalling parcllabs-1.11.2:
      Successfully uninstalled parcllabs-1.11.2
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import requests
import concurrent.futures
from parcllabs import ParclLabsClient

/Users/zach/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
api_key = os.getenv('PARCL_LABS_API_KEY')
client = ParclLabsClient(api_key, turbo_mode=True)

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": api_key
}

### 2. Leverage the V2 Prop Endpoint for the Point in Time Metrics (Aquisitions, Rental Listings and Rent Rate)

Since all of these metrics will look at data that is grouped quarterly, we can do this most efficiently by pulling all 2024 Activity for IH in one query (~24000 credits) and then analyze the resulting dataframe

In [4]:
## Construct the query for all IH Sale and Rental in 2024 using the National Parcl ID
ih_2024_query = {
    "parcl_ids": [5826765], #National Market
    "property_filters": {
        "property_types": ["SINGLE_FAMILY"],
        "include_property_details": False #Defaults to True
    },
    "event_filters": {
        "event_names": ["SOLD", "SOLD_INTER_PORTFOLIO_TRANSFER", "RENTAL_PRICE_CHANGE", "LISTED_RENT"],
        "min_event_date": "2024-01-01",
        "max_event_date": "2024-12-31"
    },
    "owner_filters": { "owner_name": ["INVITATION_HOMES"] },
}

In [5]:
## Pass in the query to the API and get the data, this will take ~24000 credits.
## We will use pagination to get all the data and then process it in the next section

try:
    # Initialize container for all events
    all_events = []
    base_url = "https://api.parcllabs.com/v2/property_search"
    limit = 1000
    
    # Function to fetch a page of data
    def fetch_page(offset):
        page_url = f"{base_url}?limit={limit}&offset={offset}"
        resp = requests.post(page_url, json=ih_2024_query, headers=headers)
        data = resp.json()
        
        if "detail" in data:
            raise ValueError(data["detail"])
            
        # Extract events with their property IDs
        events_data = []
        for prop in data.get('data', []):
            property_id = prop.get('parcl_property_id')
            for event in prop.get('events', []):
                events_data.append({"parcl_property_id": property_id, **event})
                
        return events_data, len(data.get('data', []))
    
    # Get first page and metadata
    first_page_events, first_page_props = fetch_page(0)
    all_events.extend(first_page_events)
    
    # Get total properties count for pagination
    meta_resp = requests.post(f"{base_url}?limit=1", json=ih_2024_query, headers=headers).json()
    total = meta_resp.get('metadata', {}).get('results', {}).get('total_available', 0)
    total_pages = (total + limit - 1) // limit  # Ceiling division

    print(f"Found {total} properties, fetched page 1 with {first_page_props} properties and {len(first_page_events)} events")
    print(f"Fetching {total_pages-1} remaining pages in parallel")
    
    if total_pages > 1:
        # Prepare offsets for remaining pages
        offsets = [i * limit for i in range(1, total_pages)]
        
        # Use parallel processing for remaining pages
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(10, len(offsets))) as executor:
            # Submit all page requests
            futures = {executor.submit(fetch_page, offset): offset for offset in offsets}
            
            # Process results as they complete
            for future in concurrent.futures.as_completed(futures):
                offset = futures[future]
                try:
                    page_events, page_props = future.result()
                    all_events.extend(page_events)
                    print(f"Fetched page {offset//limit + 1} with {page_props} properties and {len(page_events)} events")
                except ValueError as e:
                    print(f"Error at offset {offset}: {e}")
                except Exception as e:
                    print(f"Unexpected error at offset {offset}: {e}")
    
    # Create DataFrame
    ih_2024_df = pd.DataFrame(all_events)
    
    # Final verification
    unique_properties = ih_2024_df['parcl_property_id'].nunique()
    print(f"\nFinal results:")
    print(f"Total events collected: {len(ih_2024_df)}")
    print(f"Unique property IDs: {unique_properties}")
    print(f"Expected property count from API: {total}")
    
    # Display sample data
    display(ih_2024_df.head())
    
except ValueError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Error: {e}")

Found 24027 properties, fetched page 1 with 1000 properties and 5470 events
Fetching 24 remaining pages in parallel
Fetched page 6 with 1000 properties and 5699 events
Fetched page 8 with 1000 properties and 5715 events
Fetched page 5 with 1000 properties and 5588 events
Fetched page 9 with 1000 properties and 5724 events
Fetched page 7 with 1000 properties and 5545 events
Fetched page 4 with 1000 properties and 5599 events
Fetched page 11 with 1000 properties and 5631 events
Fetched page 2 with 1000 properties and 5658 events
Fetched page 3 with 1000 properties and 5850 events
Fetched page 10 with 1000 properties and 5334 events
Fetched page 14 with 1000 properties and 5488 events
Fetched page 16 with 1000 properties and 5455 events
Fetched page 13 with 1000 properties and 5661 events
Fetched page 12 with 1000 properties and 5654 events
Fetched page 15 with 1000 properties and 5544 events
Fetched page 19 with 1000 properties and 5384 events
Fetched page 20 with 1000 properties and 538

,parcl_property_id,event_type,event_name,event_date,entity_owner_name,true_sale_index,price,transfer_index,investor_flag,owner_occupied_flag,new_construction_flag,current_owner_flag
0,48822275,RENTAL,PRICE_CHANGE,2024-09-30,INVITATION_HOMES,3,2390.0,3,1,0,0,1
1,48822275,RENTAL,PRICE_CHANGE,2024-09-25,INVITATION_HOMES,3,2449.0,3,1,0,0,1
2,48822275,RENTAL,PRICE_CHANGE,2024-09-20,INVITATION_HOMES,3,2515.0,3,1,0,0,1
3,48822275,RENTAL,PRICE_CHANGE,2024-09-16,INVITATION_HOMES,3,2580.0,3,1,0,0,1
4,48822275,RENTAL,PRICE_CHANGE,2024-09-12,INVITATION_HOMES,3,2645.0,3,1,0,0,1


In [6]:
# Create date and quarter columns
ih_2024_df['event_date'] = pd.to_datetime(ih_2024_df['event_date'])
ih_2024_df['quarter'] = ih_2024_df['event_date'].dt.to_period('Q')

# Calculate all metrics in one go
ih_2024_quarterly_metrics = pd.DataFrame({
    'acquisition_count': ih_2024_df[ih_2024_df['event_type'] == 'SALE'].groupby('quarter')['parcl_property_id'].nunique(),
    'median_rent': ih_2024_df[ih_2024_df['event_type'] == 'RENTAL'].groupby('quarter')['price'].median(),
    'rental_listing_count': ih_2024_df[ih_2024_df['event_type'] == 'RENTAL'].groupby('quarter')['parcl_property_id'].nunique()
}).reset_index()

# Format and display
ih_2024_quarterly_metrics['quarter'] = ih_2024_quarterly_metrics['quarter'].astype(str)
ih_2024_quarterly_metrics = ih_2024_quarterly_metrics.sort_values('quarter')

ih_2024_quarterly_metrics

,quarter,acquisition_count,median_rent,rental_listing_count
0,2024Q1,239,2320.0,7688
1,2024Q2,295,2345.0,8131
2,2024Q3,556,2340.0,8197
3,2024Q4,304,2280.0,7863


### 3. Leverage the V1 Prop Endpoints for the Quarterly Inventory

Inventory is a more complex pull than just point in time metrics, because we need to know if at a given point in time whether or not that event was the latest event for the property. You can pull all events for former or curren IH homes from the V1 endpoints by passing in the csv of parcl prop IDs that have been owned by Invitation Homes at one point in their history

In [7]:
#Load list of IH Owned Parcl Prop IDs
csv_path = '/your_file_path/invitation_homes_property_ids.csv'  # Update this to your CSV path
property_df = pd.read_csv(csv_path)

parcl_property_id_list = property_df['PARCL_PROPERTY_ID'].unique().tolist()

print(f"Loaded {len(parcl_property_id_list)} unique property IDs")

Loaded 102492 unique property IDs


In [8]:
#Pass list of IH Owned Parcl Prop IDs to the V1 Endpoint
ih_owned_events = client.property.events.retrieve(
        parcl_property_ids=parcl_property_id_list,
        end_date='2024-12-31',
        event_type='SALE',

)

In [9]:
ih_owned_events

,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index,record_updated_date
0,48783765,2017-09-15,SALE,NON_ARMS_LENGTH_INTRA_PORTFOLIO_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,1,6,2,2024-12-13
1,48783765,2014-06-02,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,0,5,2,2024-12-13
2,48783765,2013-01-10,SALE,SOLD,135000.0,0.0,0,1.0,None,0,4,2,2024-12-13
3,48783765,2012-11-19,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,0.0,None,0,3,1,2024-12-13
4,48783765,2012-07-23,SALE,SOLD,106300.0,0.0,0,1.0,None,0,2,1,2024-12-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338718,197891371,2021-12-20,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,1,1.0,None,0,1,0,2024-12-13
338719,452430512,2024-11-21,SALE,SOLD_INTER_PORTFOLIO_TRANSFER,1920200.0,0.0,0,1.0,INVITATION_HOMES,1,2,1,2024-12-13
338720,452430512,2022-09-29,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,1,1.0,None,0,1,0,2024-12-13
338721,460515525,2024-05-21,SALE,SOLD,3186000.0,0.0,1,1.0,INVITATION_HOMES,1,1,1,2024-12-13


Now, with all sale events for IH props (current or former) we can backtest inventory by checking, at any point in time if Invitation Homes was the owner on the most recent sale for a property.